# Q&A against a SQL Database (Azure SQL, Azure Fabric, Synapse, SQL Managed Instance, etc)

Now that we know (from the prior Notebook) how to query tabular data on a CSV file, let's try now to keep the data at is source and ask questions directly to a SQL Database.
The goal of this notebook is to demonstrate how a LLM so advanced as GPT-4 can understand a human question and translate that into a SQL query to get the answer. 

We will be using the Azure SQL Server that you created on the initial deployment. However the same code below works with any SQL database like Synapse for example. The server should be created on the Resource Group where the Azure Cognitive Search service is located.

Let's begin..

In [8]:
%pip install -r common/requirements.txt

  Obtaining dependency information for langchain==0.0.316 from https://files.pythonhosted.org/packages/cb/58/27830eb8bdcfabc2ec9ce2fef4e9d4fda5ce7c886f6b5be18d2ee365e603/langchain-0.0.316-py3-none-any.whl.metadata
  Using cached langchain-0.0.316-py3-none-any.whl.metadata (15 kB)
  Obtaining dependency information for openai==0.28.1 from https://files.pythonhosted.org/packages/1e/9f/385c25502f437686e4aa715969e5eaf5c2cb5e5ffa7c5cdd52f3c6ae967a/openai-0.28.1-py3-none-any.whl.metadata
  Using cached openai-0.28.1-py3-none-any.whl.metadata (11 kB)
  Obtaining dependency information for tiktoken from https://files.pythonhosted.org/packages/b8/eb/234646d9eefda8a500d0fd88b05bf625a90ed18054124349db26e558276e/tiktoken-0.5.1-cp311-cp311-win_amd64.whl.metadata
  Using cached tiktoken-0.5.1-cp311-cp311-win_amd64.whl.metadata (6.8 kB)
  Obtaining dependency information for pypdf from https://files.pythonhosted.org/packages/74/a9/5ccde1312650dd03e65350224fea85d9a430c182a01f056599cbb76f7390/pypdf-3.1

ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\Python311\\Scripts\\tabulate.exe' -> 'c:\\Python311\\Scripts\\tabulate.exe.deleteme'


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
!pip install langchain --user


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for langchain from https://files.pythonhosted.org/packages/6d/8d/78e2610ac6e94813cde9c06dfce5911dd0dee6f7702921d47cc95cba4379/langchain-0.0.331-py3-none-any.whl.metadata
  Using cached langchain-0.0.331-py3-none-any.whl.metadata (16 kB)
  Obtaining dependency information for aiohttp<4.0.0,>=3.8.3 from https://files.pythonhosted.org/packages/2e/9f/9c37b01fc6a37c92f139a4cd937a92f03ebbd75379cfd55e85ca1e571643/aiohttp-3.8.6-cp311-cp311-win_amd64.whl.metadata
  Using cached aiohttp-3.8.6-cp311-cp311-win_amd64.whl.metadata (7.9 kB)
  Obtaining dependency information for anyio<4.0 from https://files.pythonhosted.org/packages/19/24/44299477fe7dcc9cb58d0a57d5a7588d6af2ff403fdd2d47a246c91a3246/anyio-3.7.1-py3-none-any.whl.metadata
  Using cached anyio-3.7.1-py3-none-any.whl.metadata (4.7 kB)
  Obtaining dependency information for dataclasses-json<0.7,>=0.5.7 from https://files.pythonhosted.org/packages/21/1f/1cff009cff64420572b9f75b70e4a054095719179a172297dfdd6


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
%pip install dotenv --user

  Using cached dotenv-0.0.5.tar.gz (2.4 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [70 lines of output]
      c:\Python311\Lib\site-packages\setuptools\installer.py:27: SetuptoolsDeprecationWarning: setuptools.installer is deprecated. Requirements should be satisfied by a PEP 517 installer.
        warnings.warn(
        error: subprocess-exited-with-error
      
        Ã— python setup.py egg_info did not run successfully.
        â”‚ exit code: 1
        â•°â”€> [17 lines of output]
            Traceback (most recent call last):
              File "<string>", line 2, in <module>
              File "<pip-setuptools-caller>", line 14, in <module>
              File "C:\Users\davyu\AppData\Local\Temp\pip-wheel-qtvh7_vc\distribute_3dcc99aca20a4ed680789ca2221dd852\setuptools\__init__.py", line 2, in <module>
                from setuptools.extension import Extension, Library
              File "C:\Users\davyu\AppData\Local\Temp\pip-wheel-qtvh7_vc\distri

In [18]:
import os
import pandas as pd
import pyodbc
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.sql_database import SQLDatabase
from langchain.agents import AgentExecutor
from langchain.callbacks.manager import CallbackManager

from common.prompts import MSSQL_PROMPT, MSSQL_AGENT_PREFIX, MSSQL_AGENT_FORMAT_INSTRUCTIONS

from IPython.display import Markdown, HTML, display  

from dotenv import load_dotenv
load_dotenv("credentials.env", override=True)


def printmd(string):
    display(Markdown(string))

ModuleNotFoundError: No module named 'dotenv'

In [ ]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_GPT4_ENDPOINT"]
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_GPT4_KEY"]
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]
os.environ["OPENAI_API_TYPE"] = "azure"

MODEL = os.environ[ "AZURE_OPENAI_GPT4_DEPLOYMENT" ]

# Install MS SQL DB driver in your machine

We need the driver installed on this compute in order to talk to the SQL DB, so run the below cell once<br>
Reference: https://learn.microsoft.com/en-us/sql/connect/odbc/linux-mac/installing-the-microsoft-odbc-driver-for-sql-server?view=sql-server-ver16&tabs=ubuntu18-install%2Calpine17-install%2Cdebian8-install%2Credhat7-13-install%2Crhel7-offline

In [ ]:
# !sudo ./download_odbc_driver.sh

# Load Azure SQL DB with the Covid Tracking CSV Data

The Azure SQL Database is currently empty, so we need to fill it up with data. Let's use the same data on the Covid CSV filed we used on the prior Notebook, that way we can compare results and methods. 
For this, you will need to type below the credentials you used at creation time.

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.engine.url import URL

db_config = {
                'drivername': 'mssql+pyodbc',
                'username': os.environ["SQL_SERVER_USERNAME"] +'@'+ os.environ["SQL_SERVER_NAME"],
                'password': os.environ["SQL_SERVER_PASSWORD"],
                'host': os.environ["SQL_SERVER_NAME"],
                'port': 1433,
                'database': os.environ["SQL_SERVER_DATABASE"],
                'query': {'driver': 'ODBC Driver 17 for SQL Server'}
            }

# Create a URL object for connecting to the database
db_url = URL.create(**db_config)

# Print the resulting URL string
# print(db_url)

# Connect to the Azure SQL Database using the URL string
engine = create_engine(db_url)

# Test the connection
try:
    conn = engine.connect()
    print("Connection successful!")
    result = engine.execute("SELECT @@Version")
    for row in result:
        print(row)
    conn.close()
    
except OperationalError:
    print("Connection failed.")

In [ ]:
# Read CSV file into a pandas dataframe
csv_path = "./data/all-states-history.csv"
df = pd.read_csv(csv_path).fillna(value = 0)

# Infer column names and data types
column_names = df.columns.tolist()
column_types = df.dtypes.to_dict()

# Generate SQL statement to create table
table_name = 'covidtracking'

create_table_sql = f"CREATE TABLE {table_name} ("
for name, dtype in column_types.items():
    if dtype == 'object':
        create_table_sql += f"{name} VARCHAR(MAX), "
    elif dtype == 'int64':
        create_table_sql += f"{name} INT, "
    elif dtype == 'float64':
        create_table_sql += f"{name} FLOAT, "
    elif dtype == 'bool':
        create_table_sql += f"{name} BIT, "
    elif dtype == 'datetime64[ns]':
        create_table_sql += f"{name} DATETIME, "
create_table_sql = create_table_sql[:-2] + ")"

try:
    #Createse the table in Azure SQL
    engine.execute(create_table_sql)
    print("Table",table_name,"succesfully created")
    # Insert data into SQL Database
    lower = 0
    upper = 1000
    limit = df.shape[0]

    while lower < limit:
        df[lower:upper].to_sql(table_name, con=engine, if_exists='append', index=False)
        print("rows:", lower, "-", upper, "inserted")
        lower = upper
        upper = min(upper + 1000, limit)

except Exception as e:
    print(e)

# Query with LLM

**Note**: We are here using Azure SQL, however the same code will work with Synapse, SQL Managed instance, or any other SQL engine. You just need to provide the right ENV variables and it will connect succesfully.

In [ ]:
# Create or LLM Langchain object using GPT-4 deployment
# Again we need GPT-4. It is necesary in the use of Agents. GPT-35-Turbo will make many mistakes.
llm = AzureChatOpenAI(deployment_name=MODEL, temperature=0, max_tokens=500)

In [ ]:
# Let's create the db object
db = SQLDatabase.from_uri(db_url)

In [ ]:
# Natural Language question (query)
QUESTION = "How may patients were hospitalized during July 2020 in Texas, and nationwide as the total of all states? Use the hospitalizedIncrease column"

### SQL Agent

Let's use an agent now and see how ReAct framework solves the problem.

In [ ]:
toolkit = SQLDatabaseToolkit(db=db, llm=llm)

agent_executor = create_sql_agent(
    prefix=MSSQL_AGENT_PREFIX,
    format_instructions = MSSQL_AGENT_FORMAT_INSTRUCTIONS,
    llm=llm,
    toolkit=toolkit,
    top_k=30,
    verbose=True
)

In [ ]:
# As we know by now, Agents use expert/tools. Let's see which are the tools for this SQL Agent
agent_executor.agent.allowed_tools

In [ ]:
# And let's see now our clever crafted prompt
printmd(agent_executor.agent.llm_chain.prompt.template)

In [ ]:
for i in range(2):
    try:
        response = agent_executor.run(QUESTION) 
        break
    except Exception as e:
        response = str(e)
        continue

In [ ]:
printmd(response)

**IMPORTANT NOTE**: If you don't specify the column name on the question, runing the above cell multiple times will yield diferent results some times. <br>
The reason is:
The column names are ambiguous, hence it is hard even for Humans to discern what are the right columns to use

# Summary

In this notebook, we achieved our goal of Asking a Question in natural language to a dataset located on a SQL Database.  We did this by using purely prompt engineering (Langchain does it for us) and the cognitive power of GPT-4.

This process shows why it is NOT necessary to move the data from its original source as long as the source has an API and a common language we can use to interface with. GPT-4 has been trained on the whole public Github corpus, so it can pretty much understand most of the coding and database query languages that exists out there. 

# NEXT

The Next Notebook will show you how to create a custom REACT agent that connects to the internet using BING SEARCH API to answer questions grounded on search results with citations. Basically a clone of Bing Chat.